# LangChain의 RAG 파헤치기

![rag-1.png](./assets/rag-1.png)

![rag-2.png](./assets/rag-2.png)

### 1. 질문 처리

질문 처리 단계에서는 사용자의 질문을 받아 이를 처리하고, 관련 데이터를 찾는 작업이 이루어집니다. 이를 위해 다음과 같은 구성 요소들이 필요합니다:

- **데이터 소스 연결**: 질문에 대한 답변을 찾기 위해 다양한 텍스트 데이터 소스에 연결해야 합니다. LangChain은 다양한 데이터 소스와의 연결을 간편하게 설정할 수 있도록 돕습니다.
- **데이터 인덱싱 및 검색**: 데이터 소스에서 관련 정보를 효율적으로 찾기 위해, 데이터는 인덱싱되어야 합니다. LangChain은 인덱싱 과정을 자동화하고, 사용자의 질문과 관련된 데이터를 검색하는 데 필요한 도구를 제공합니다.

### 2. 답변 생성

관련 데이터를 찾은 후에는 이를 기반으로 사용자의 질문에 답변을 생성해야 합니다. 이 단계에서는 다음 구성 요소가 중요합니다:

- **답변 생성 모델**: LangChain은 고급 자연어 처리(NLP) 모델을 사용하여 검색된 데이터로부터 답변을 생성할 수 있는 기능을 제공합니다. 이러한 모델은 사용자의 질문과 검색된 데이터를 입력으로 받아, 적절한 답변을 생성합니다.


# RAG for KBO 2024

야구를 주제로 한 RAG(Retrieval-Augmented Generation) 시스템 구축 실습입니다.
KBO 2025 레코드북과 연감 데이터를 사용하여 질문에 답변하는 시스템을 만들어 봅니다.

## 1. 환경 설정

API 키 등 필요한 환경 변수를 로드합니다.

In [5]:
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수 로드
load_dotenv()

# LangSmith 설정 (선택 사항)
# os.environ['LANGCHAIN_TRACING_V2'] = 'true'
# os.environ['LANGCHAIN_API_KEY'] = 'your-api-key'

True

## 2. 문서 로드 (Load Documents)

baseball 폴더에 있는 PDF 파일들을 로드합니다.

In [6]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

# baseball 폴더 내의 모든 pdf 파일 로드
loader = DirectoryLoader("./baseball", glob="*.pdf", loader_cls=PyPDFLoader)
docs = loader.load()

print(f"총 {len(docs)} 페이지의 문서가 로드되었습니다.")
print(f"첫 번째 문서의 일부 내용:\n{docs[0].page_content[:200]}")

Ignoring wrong pointing object 2 65536 (offset 0)
Ignoring wrong pointing object 9 65536 (offset 0)
Ignoring wrong pointing object 16 65536 (offset 0)
Ignoring wrong pointing object 39 65536 (offset 0)
Ignoring wrong pointing object 45 65536 (offset 0)
Ignoring wrong pointing object 93 65536 (offset 0)
Ignoring wrong pointing object 120 65536 (offset 0)
Ignoring wrong pointing object 136 65536 (offset 0)
Ignoring wrong pointing object 143 65536 (offset 0)
Ignoring wrong pointing object 168 65536 (offset 0)
Ignoring wrong pointing object 174 65536 (offset 0)
Ignoring wrong pointing object 200 65536 (offset 0)
Ignoring wrong pointing object 206 65536 (offset 0)
Ignoring wrong pointing object 212 65536 (offset 0)
Ignoring wrong pointing object 225 65536 (offset 0)
Ignoring wrong pointing object 231 65536 (offset 0)
Ignoring wrong pointing object 237 65536 (offset 0)
Ignoring wrong pointing object 256 65536 (offset 0)
Ignoring wrong pointing object 280 65536 (offset 0)
Ignoring wrong point

총 960 페이지의 문서가 로드되었습니다.
첫 번째 문서의 일부 내용:



In [7]:
print(f"첫 번째 문서의 일부 내용:\n{docs[250].page_content[:200]}")
print(f"첫 번째 문서의 일부 내용:\n{docs[510].page_content[:200]}")
print(f"두 번째 문서의 일부 내용:\n{docs[850].page_content[:200]}")
print(f"두 번째 문서의 일부 내용:\n{docs[900].page_content[:200]}")

첫 번째 문서의 일부 내용:
249김정수 (롯) 49 김우열 (O) 56
이종도 (M) 48 윤동균 (O) 54
윤동균 (O) 47 김용희 (롯) 52
김준환 (해) 45 신경식 (O) 50
1984 --------------- 1985 ---------------
이만수 (삼) 80 이만수 (삼) 87
이광은 (M) 68 김성한 (해) 75
김용철 (롯) 67 장효조 (삼) 65
김
첫 번째 문서의 일부 내용:
509연도 참가 구단총
경기수팀당
경기수팀간
경기수시즌구분경기제도 우승팀 비고
1992(8개 구단)
OB 베어스해태 타이거즈
삼성 라이온즈
롯데 자이언츠빙그레 이글스
태평양 돌핀스
LG 트윈스
쌍방울 레이더스504 126 18
〃 〃정규시즌1위-빙그레2위-해태
3위-롯데
4위-삼성준PO-롯데
PO 진출
PO-롯데시리즈 진출
한국시리즈
-롯데
1993
〃50
두 번째 문서의 일부 내용:
홈․런․기․록
325연도별 10홈런 선점 선수
<일자 기준 / 선점과 최소 경기 다른 경우 별도 표시>
연도 선수명(팀) 일자 개인 경기수 팀 경기수 달성당시 연령
1982 김우열(O) 5.23 22 24 3 2 세   8 개 월  1 4 일
1983 김봉연(해) 5.21 25 25 3 1 세   4 개 월   8 일
1984 이만수(삼) 5.10 24 2
두 번째 문서의 일부 내용:
끝․내․기․기․록
375구분 일자 대진 구장 이닝 점수 아웃 주자 타자명 투수명 볼카운트
4 9 2 0 1 4 . 8 .2 9 넥   센  -  한   화 대  전 1 09-9
9-101사
123  정 범 모  송 신 영 3  -  1
5 0 2 0 1 5 . 5 .1 7 넥   센  -  한   화 대  전 1 06-6
6-72사
123  강 경 학  배


## 3. 텍스트 분할 (Split Documents)

로드된 문서를 작은 청크(chunk)로 분할합니다.
RecursiveCharacterTextSplitter를 사용합니다.

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=100
)

splits = text_splitter.split_documents(docs)
print(f"총 {len(splits)} 개의 청크로 분할되었습니다.")

총 3044 개의 청크로 분할되었습니다.


## 4. 벡터 스토어 생성 (Create Vector Store)

FAISS를 사용하여 벡터 스토어를 생성합니다.

In [9]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# Embeddings 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

#  첫 배치만 from_documents로 인덱스 생성
FIRST_BATCH_SIZE = 30
first_batch = splits[:FIRST_BATCH_SIZE]

vectorstore = FAISS.from_documents(first_batch, embeddings)
print("초기 vectorstore 생성 완료")

#  나머지 문서들을 add_documents로 안전하게 추가
BATCH_SIZE = 50

for i in range(FIRST_BATCH_SIZE, len(splits), BATCH_SIZE):
    batch = splits[i : i + BATCH_SIZE]
    print(f"배치 추가 중: {i} ~ {i+len(batch)}")
    vectorstore.add_documents(batch)

print("모든 청크가 벡터스토어에 성공적으로 추가됨")

# 저장
vectorstore.save_local("./faiss_index")
print("FAISS index 저장 완료")

초기 vectorstore 생성 완료
배치 추가 중: 30 ~ 80
배치 추가 중: 80 ~ 130
배치 추가 중: 130 ~ 180
배치 추가 중: 180 ~ 230
배치 추가 중: 230 ~ 280
배치 추가 중: 280 ~ 330
배치 추가 중: 330 ~ 380
배치 추가 중: 380 ~ 430
배치 추가 중: 430 ~ 480
배치 추가 중: 480 ~ 530
배치 추가 중: 530 ~ 580
배치 추가 중: 580 ~ 630
배치 추가 중: 630 ~ 680
배치 추가 중: 680 ~ 730
배치 추가 중: 730 ~ 780
배치 추가 중: 780 ~ 830
배치 추가 중: 830 ~ 880
배치 추가 중: 880 ~ 930
배치 추가 중: 930 ~ 980
배치 추가 중: 980 ~ 1030
배치 추가 중: 1030 ~ 1080
배치 추가 중: 1080 ~ 1130
배치 추가 중: 1130 ~ 1180
배치 추가 중: 1180 ~ 1230
배치 추가 중: 1230 ~ 1280
배치 추가 중: 1280 ~ 1330
배치 추가 중: 1330 ~ 1380
배치 추가 중: 1380 ~ 1430
배치 추가 중: 1430 ~ 1480
배치 추가 중: 1480 ~ 1530
배치 추가 중: 1530 ~ 1580
배치 추가 중: 1580 ~ 1630
배치 추가 중: 1630 ~ 1680
배치 추가 중: 1680 ~ 1730
배치 추가 중: 1730 ~ 1780
배치 추가 중: 1780 ~ 1830
배치 추가 중: 1830 ~ 1880
배치 추가 중: 1880 ~ 1930
배치 추가 중: 1930 ~ 1980
배치 추가 중: 1980 ~ 2030
배치 추가 중: 2030 ~ 2080
배치 추가 중: 2080 ~ 2130
배치 추가 중: 2130 ~ 2180
배치 추가 중: 2180 ~ 2230
배치 추가 중: 2230 ~ 2280
배치 추가 중: 2280 ~ 2330
배치 추가 중: 2330 ~ 2380
배치 추가 중: 2380 ~ 2430
배치 추가 중: 2430

## 5. 검색기(Retriever) 설정 및 확인

Retriever를 설정하고, 실제 질문에 대해 어떤 문서가 검색되는지 확인합니다.

In [ ]:
# 검색기 설정 (k=5, similarity)
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

# 검색기 작동 확인
print("검색기 작동 확인!!")
docs = retriever.invoke("700승을 기록한 감독은 누구인가요")
print(f"검색된 문서 개수: {len(docs)}")
for i, doc in enumerate(docs):
    print(f"\n[문서 {i+1}]")
    print(doc.page_content[:200])

검색기 작동 확인!!
검색된 문서 개수: 5

[문서 1]
기․타․기․록
424
감독 700승
■ 1993년 9월 12일 해태 김응용, 빙그레 김영덕 감독은 같은 날 700승을 달성했으며, 김영덕 감독은 1,1 96경기로 역
대 최소 경기 700승 기록을 가지고 있다. 이후 감독 700승을 달성한 감독은 김성근, 강병철, 김인식, 김재박 , 김경문 
감독이 있으며, 2024년에는 2시즌 만에 현역으로 복귀한 김태형

[문서 2]
8 2024. 8.31 김태형(롯) 1,269 두  산 잠 실 56세 11개월 19일
감독 1,000승
■ 해태 시절 통산 9차례나 팀을 우승으로 이끌었던 김응용 감독은 1998년 6월 7일 무등에서 
삼성을 상대로 누구도 달성하지 못한 감독 1,000승이라는 대기록을 세웠다. 통산 1,554승을 
기록한 김응용 감독의 뒤를 이어 김성근 감독이 1,388승

[문서 3]
(야쿠르트)이 기록한 3,248경기이다.
구분 감독명(팀) 일자 상대팀(구장) 승 패 무 달성당시 연령
1 김응용(삼) 2003. 9. 8 현  대(수원) 1,382 1,063 55 61세 11개월 24일
2 김성근(한) 2016. 5. 1 삼  성(대전) 1,313 1,130 57 73세  4개월 18일
감독 500승
구분 일자 감독명(팀) 경기수 상대팀

[문서 4]
117연도별(구단별) 감독 ․코치 및 선수 등록 현황
※ 2.1 소속선수 등록일 기준
구단 1982 1983 1984 1985 1986 1987 1988 1989
감독 코치 선수 소계 감독 코치 선수 소계 감독 코치 선수 소계 감독 코치 선수 소계 감독 코치 선수 소계 감독 코치 선수 소계 감독 코치 선수 소계 감독 코치 선수 소계
해  태 1 2 22 2

[문서 5]
기․타․기․록
423감독 2,500경기 출장
■ 삼성 김응용 감독은 2003년 9월 8일 수원 현대전에서 지난 2000년 프로 최초의 감독 2,000경기 출장에  이어 2,500
경기 출장이라는 또 하나의 대업을 달성했다. 1983년 

## 6. 프롬프트(Prompt) 설정 및 확인

RAG에 사용할 프롬프트를 로드하고 확인합니다.

In [11]:
from langchain import hub

# 프롬프트 로드
prompt = hub.pull("rlm/rag-prompt")

# 프롬프트 내용 출력
print("\n프롬프트 템플릿:")
print(prompt.messages[0].prompt.template)


프롬프트 템플릿:
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


## 7. RAG 체인 생성 (Create RAG Chain)

문서 검색기(Retriever), 프롬프트, LLM을 연결하여 RAG 체인을 만듭니다.

In [13]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

# LLM 설정 (gpt-5.1)
llm = ChatOpenAI(model_name="gpt-5.1", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# RAG 체인 구성
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## 8. 질문하기

KBO 규정이나 기록에 대한 질문을 던져봅니다.

In [17]:
# 질문
question = "40 세이브를 최초로 달성한 선수는 누구인가요"
# question = "시즌 최다 세이브를 달성한 선수는 누구인가요. 그리고 총 몇세이브를 달성했나요"
# ㅂuestion = "700승을 기록한 감독은 누구인가요"


# 답변 생성
response = rag_chain.invoke(question)


print(f"질문: {question}\n")
print(f"답변: {response}")

질문: 40 세이브를 최초로 달성한 선수는 누구인가요

답변: KBO 리그에서 한 시즌 40세이브를 최초로 달성한 선수는 태평양 돌핀스의 정명원입니다.
